In [ ]:
# -*- coding: cp1252 -*-

#QUESTA LIBRERIA E' FRUTTO DELLA COLLABORAZIONE FRA DATAREPORTER E RETESVILUPPO SC
#DATAREPORTER http://www.datareporter.it/
#RETESVILUPPO: http://www.retesviluppo.it
#IN CASO DI UTILIZZO SI PREGA DI CITARE LE DUE STRUTTURE
#AUTORE: MARCO SCARSELLI

from mechanize import Browser
import os
import pandas as pd
import html5lib
from bs4 import BeautifulSoup  
from python_geojson_pandas_join import join


#os.chdir(r"E:\Dropbox\Dropbox\0_DB\A_Elaborazioni\Coeweb")

class coeweb(object):
    
    def __init__(self, link):
        self.pandas_db = pd.DataFrame()
        self.link = link
        br = Browser()
        br.open(self.link)
        br.select_form(nr=0)
        
        self.paesi = []
        #restituisce la lista di paesi / macro aree mondiali di export
        self.paesi_descizione = []
        control = br.form.find_control("PAESE")
        if control.type == "select":  
            for item in control.items:
                self.paesi.append(item.name)
                self.paesi_descizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.territori = []
        #restituisce la lista delle territori italiani presenti in coeweb
        self.territori_descrizione = []
        control = br.form.find_control("CODTERRITORIO")
        if control.type == "select":  
            for item in control.items:
                self.territori.append((item.name, str([label.text  for label in item.get_labels()])))
                self.territori_descrizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.anni = []
        control = br.form.find_control("ANNO")
        if control.type == "select":  
            for item in control.items:
                self.anni.append((item.name, str([label.text  for label in item.get_labels()])))
        self.mese = []
        control = br.form.find_control("MESE")
        if control.type == "select": 
            for item in control.items:
                self.mese.append((item.name, str([label.text  for label in item.get_labels()])))
        self.cumulato = []
        control = br.form.find_control("TIPOVALORE")
        for item in control.items:
            self.cumulato.append((item.name, str([label.text  for label in item.get_labels()])))
        
        #esempio di raggruppamento di paesi verso cui si effettua export predefinito dentro la classe 
        self.bric = ['508', '720', '664', '75']
        #[('508', "['508 - Brasile']"), ('720', "['720 - Cina']"), ('664', "['664 - India']"), ('75', "['75 - Russia']")]
    
    def convert_string_euro(self,x):
        '''questa funzione rimuove la formattazione dei numeri di coeweb, trasformando il testo in numeri'''
        try: 
            return float(x.replace(".", ""))
        except:
            pass
    

    def scarica_uno(self, territorio = ["309048"], paese = ["1033"], cumulato = ['M'], anno = ["2013"], mese = ["4"]  ):# , paese, anno, trimestre, modalita)
        '''
        effettua il download dell'export di un solo territorio italiano verso un solo paese
        '''
        br = Browser()
        br.open(self.link)

        br.select_form(nr=0)

        control_territorio = br.form.find_control("CODTERRITORIO")
        br[control_territorio.name] = territorio #["309048"]

        control_paese = br.form.find_control("PAESE")
        br[control_paese.name] = paese #["309048"]

        control_anno = br.form.find_control("ANNO")
        br[control_anno.name] = anno #["309048"]

        control_mese = br.form.find_control("MESE")
        br[control_mese.name] = mese #["309048"]

        control_cumulato = br.form.find_control("TIPOVALORE")
        br[control_cumulato.name] = cumulato #["309048"]

        nome_stringa = territorio[0] + "_" + paese[0] + "_" + anno[0] + "_" + mese[0] + "_" + cumulato[0]


        response = br.submit()
        out_file = open(os.path.join(os.getcwd(), "scaricati", nome_stringa +".html"), "w")
        out_file.write(response.read()) 
        out_file.close()
        #os.system("start " + nome_stringa +".html")
        self.dati = pd.read_html(os.path.join(os.getcwd(), "scaricati", nome_stringa +".html"), header= 0, index_col=0, encoding = "cp1252")#'utf8'
        #os.remove(territorio[0] +".html")
        print "download_" + nome_stringa
        br.close()
        return self.dati
        
        # i dati dovrebbero essere raccolti in un pandas 
        
    def scarica_molti(self, territorio = ["309048"],   lista_paesi = None, lista_anni = None, lista_mesi = None, lista_cumulato = ["M"]):
        ''' questa funzione permette di scaricare per uno o più territori uno o più settori, per uno o più periodi di tempo'''
        #self.pandas_db = pd.DataFrame()
        
        def nome_paese(dati, codice):
            for item in dati:
                if item[0] == codice:
                    pulito1= item[1].split("'",)[1]
                    pulito2 = pulito1.split(" - ",)[1]
                    print pulito2
                    return pulito2
        
        def nome_territorio(dati, codice):
            for item in dati:
                if item[0] == codice:
                    try:
                        pulito1= item[1].split("'")[1]
                        pulito1 = pulito1.replace("-","")
                        pulito1 = pulito1.replace("'","")
                        return pulito1
                    except:
                        pass
        
        
        for item_territori in territorio:
            for item_paesi in lista_paesi:
                for item_anni in lista_anni:
                    for item_mesi in lista_mesi:

                        dataset =  self.scarica_uno([item_territori], paese = [str(item_paesi)],  anno = [str(item_anni)], mese = [str(item_mesi)], cumulato = lista_cumulato)
                        dataset = dataset[1]#.set_index(territorio, str(item_paesi), str(item_anni), str(item_mesi)])
                        
                        
                        dataset["territorio"] = nome_territorio(self.territori_descrizione,  item_territori)  
                        #dataset["territorio"] = item_territori
                        
                        #nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["paese"] = nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["anno"] = str(item_anni)
                        dataset["mese"] = str(item_mesi)
                        dataset["cumulato"] = lista_cumulato[0]
                        dataset.set_index(["territorio","paese","anno","mese","cumulato"], append = True, inplace = True)
                        dataset["DATI EXPORT"] = dataset["EXP" + str(item_anni)]
                          
                        
                        
                        dataset = dataset["DATI EXPORT"].apply(lambda x: self.convert_string_euro(x))
                        
                        
                        
                        # datasetcompleto
                        self.pandas_db = pd.concat([self.pandas_db, pd.DataFrame(dataset)])
                        # settori ateco disponibili
                        self.pandas_db.settori = set(self.pandas_db.reset_index()["Divisioni"])
                        
                            #print "errore_" + territorio[0] + "_" + str(item_paesi) + "_" + str(item_anni) + "_" + str(item_mesi) + "_" + lista_cumulato[0]

        for scaricato in os.listdir(os.path.join(os.getcwd(), "scaricati")):
            os.remove(os.path.join(os.getcwd(), "scaricati", scaricato))
        

        print "eseguito"
    
    
    
    def analizza_settori(self, settori, interpolazione = "no"):
        ''' questa funzione restituisce un dataframe e un dizionario contenente le serie storiche per i settori inseriti come argomento [lista]. 
        Effettua anche l'interpolazione dei dati mancanti'''
        self.dict_settore = {}
        for item in settori:
            dati_settore = self.pandas_db.xs(item, level=0)
            if interpolazione == "yes":
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
            else:
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x))
            self.dict_settore[item] = dati_settore["DATI EXPORT"]
        self.dataframe_settori = pd.DataFrame(self.dict_settore)
        return self.dataframe_settori

In [5]:
def convert_string_euro(x):
    try: 
        return float(x.replace(".", ""))
    except:
        pass

In [68]:
# ESEMPIO DI UTILIZZO: RECUPERO DI EXPORT VERSO UK E TOTALE da ISTAT COEWEB E VISUALIZZAZIONE VALORE ASSOLUTO E RAPPORTO UK SU TOTALE

In [6]:
# ESEMPIO DI UTILIZZO: si istanzia un oggetto 
Italia = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [7]:
# si aggiungono tutte le province italiane per un futuro utilizzo nella funzione
italia_province = []
for item in Italia.territori:
    if len(item[0]) == 6:
        italia_province.append(item[0])

In [8]:
#esempio di scaricamento dell'export di tutte le province italiane verso Mondo e Gran Bretagna
Italia.scarica_molti(territorio = italia_province, lista_paesi = ['6', '1033'], lista_anni = ["2015"], lista_mesi = ["9"], lista_cumulato = ['C'])

download_101001_6_2015_9_C
Regno Unito
download_101001_1033_2015_9_C
[MONDO]
download_101002_6_2015_9_C
Regno Unito
download_101002_1033_2015_9_C
[MONDO]
download_101003_6_2015_9_C
Regno Unito
download_101003_1033_2015_9_C
[MONDO]
download_101004_6_2015_9_C
Regno Unito
download_101004_1033_2015_9_C
[MONDO]
download_101005_6_2015_9_C
Regno Unito
download_101005_1033_2015_9_C
[MONDO]
download_101006_6_2015_9_C
Regno Unito
download_101006_1033_2015_9_C
[MONDO]
download_101096_6_2015_9_C
Regno Unito
download_101096_1033_2015_9_C
[MONDO]
download_101103_6_2015_9_C
Regno Unito
download_101103_1033_2015_9_C
[MONDO]
download_102007_6_2015_9_C
Regno Unito
download_102007_1033_2015_9_C
[MONDO]
download_103012_6_2015_9_C
Regno Unito
download_103012_1033_2015_9_C
[MONDO]
download_103013_6_2015_9_C
Regno Unito
download_103013_1033_2015_9_C
[MONDO]
download_103014_6_2015_9_C
Regno Unito
download_103014_1033_2015_9_C
[MONDO]
download_103015_6_2015_9_C
Regno Unito
download_103015_1033_2015_9_C
[MONDO]

In [9]:
#restituzione in formato DataFrame di Pandas
Italia_Export_Regno_Unito_Totale = Italia.pandas_db

In [10]:
Italia_Export_Regno_Unito_Totale

,,,,,,DATI EXPORT
Divisioni,territorio,paese,anno,mese,cumulato,
"AA01-Prodotti agricoli, animali e della caccia",Torino,Regno Unito,2015,9,C,1.008820e+05
AA02-Prodotti della silvicoltura,Torino,Regno Unito,2015,9,C,0.000000e+00
AA03-Prodotti della pesca e dell'acquacoltura,Torino,Regno Unito,2015,9,C,1.500000e+01
BB06-Petrolio greggio e gas naturale,Torino,Regno Unito,2015,9,C,0.000000e+00
BB07-Minerali metalliferi,Torino,Regno Unito,2015,9,C,0.000000e+00
BB08-Altri minerali da cave e miniere,Torino,Regno Unito,2015,9,C,7.486790e+05
CA10-Prodotti alimentari,Torino,Regno Unito,2015,9,C,4.050839e+07
CA11-Bevande,Torino,Regno Unito,2015,9,C,1.387200e+07
CA12-Tabacco,Torino,Regno Unito,2015,9,C,3.050000e+02


In [18]:
#AGGANCIO DEI DATI AD UNA MAPPA GEOJSON
key = pd.read_excel("aggancio_nuts_ok.xlsx")
Italia_Export_Regno_Unito_Totale.reset_index(inplace = True)

In [19]:
Italia_Export_Regno_Unito_Totale_NUTS = pd.merge(Italia_Export_Regno_Unito_Totale, key, left_on = "territorio", right_on ="COEWEB")

In [21]:
#salvataggio in excel
Italia_Export_Regno_Unito_Totale_NUTS.to_excel("Export_provinciale_verso_UK_MONDO.xlsx")

In [29]:
Italia_Export_Regno_Unito_Totale_NUTS.set_index([  u'Divisioni',  u'territorio',       u'paese',        u'anno',
              u'mese',    u'cumulato',  u'COEWEB',
              u'NUTS',    u'NUTS_MAP',    u'EUROSTAT'], inplace = True)

In [31]:
Italia_Export_Regno_Unito_Totale_NUTS

,,,,,,,,,,DATI EXPORT
Divisioni,territorio,paese,anno,mese,cumulato,COEWEB,NUTS,NUTS_MAP,EUROSTAT,
"AA01-Prodotti agricoli, animali e della caccia",Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,100882.0
AA02-Prodotti della silvicoltura,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,0.0
AA03-Prodotti della pesca e dell'acquacoltura,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,15.0
BB06-Petrolio greggio e gas naturale,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,0.0
BB07-Minerali metalliferi,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,0.0
BB08-Altri minerali da cave e miniere,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,748679.0
CA10-Prodotti alimentari,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,40508394.0
CA11-Bevande,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,13871997.0
CA12-Tabacco,Torino,Regno Unito,2015,9,C,Torino,ITC11,ITC11,Torino,305.0


In [62]:
#creo una mappa provnciale che mostra l'export verso UK in valore assoluto per provincia
UK
UK_totale = UK.xs('Totale', level='Divisioni')
UK_totale.reset_index(inplace = True)

link_geojson = os.path.join(os.getcwd(), "geojson", "province2010_2006.geojson") 
link_output = os.path.join(os.getcwd(), "uk_totale.geojson")
print join(link_geojson, UK_totale, u'nuts_id', u'NUTS_MAP', link_output, missing_columns=2, type_dataframe = True)


executed e:\Dropbox\Dropbox\0_DB\coeweb\geojson\province2010_2006.geojson dataframe


In [37]:
#creo una mappa provnciale che mostra la percentuale di export verso UK sul totale per provincia
UK = Italia_Export_Regno_Unito_Totale_NUTS.xs('Regno Unito', level='paese')
Mondo = Italia_Export_Regno_Unito_Totale_NUTS.xs('[MONDO]', level='paese')

rapport_UK_Mondo = UK / Mondo * 100


In [52]:
rapport_UK_Mondo_Nace_Totale = rapport_UK_Mondo.xs('Totale', level='Divisioni')
rapport_UK_Mondo_Nace_Totale.reset_index(inplace = True)
rapport_UK_Mondo_Nace_Totale

Index([ u'territorio',        u'anno',        u'mese',    u'cumulato',
            u'COEWEB',        u'NUTS',    u'NUTS_MAP',    u'EUROSTAT',
       u'DATI EXPORT'],
      dtype='object')

In [40]:
#la mia libreria mappa solo dataframe monodimensionali

link_geojson = os.path.join(os.getcwd(), "geojson", "province2010_2006.geojson") 
link_output = os.path.join(os.getcwd(), "output.geojson")
print join(link_geojson, rapport_UK_Mondo_Nace_Totale, u'nuts_id', u'NUTS_MAP', link_output, missing_columns=2, type_dataframe = True)

python_geojson_pandas_join.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe.dropna(inplace=True, axis=1, thresh=missing_columns)
python_geojson_pandas_join.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe.dropna(inplace=True, axis=0, thresh=missing_rows)
c:\Users\SANTECH\Anaconda\lib\site-packages\pandas\core\frame.py:2756: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


executed e:\Dropbox\Dropbox\0_DB\coeweb\geojson\province2010_2006.geojson dataframe


In [67]:
from IPython.display import HTML
HTML('<iframe width="100%" height="520" frameborder="0" src="https://scarsellirs.cartodb.com/viz/9272eb86-3ab1-11e6-9a6b-0e3a376473ab/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')

In [56]:
#ho inserito il geojson in cartodb

In [61]:
from IPython.display import HTML
HTML('<iframe width="100%" height="520" frameborder="0" src="https://scarsellirs.cartodb.com/viz/0c3aed30-3ab0-11e6-85ed-0ef7f98ade21/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')